In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from pyvbmc import VBMC
import corner
from tqdm.notebook import tqdm
import pickle
import random
from scipy.integrate import cumulative_trapezoid as cumtrapz

from time_vary_norm_utils import (
    up_or_down_RTs_fit_fn, cum_pro_and_reactive_time_vary_fn,
    rho_A_t_VEC_fn, up_or_down_RTs_fit_wrt_stim_fn, rho_A_t_fn, cum_A_t_fn,
    CDF_E_minus_small_t_NORM_rate_norm_l_time_varying_fn, rho_E_minus_small_t_NORM_rate_norm_time_varying_fn)
from types import SimpleNamespace
from time_vary_and_norm_simulators import psiam_tied_data_gen_wrapper_rate_norm_fn

In [3]:
exp_df = pd.read_csv('../outExp.csv')

# remove wrong rows 
count = ((exp_df['RTwrtStim'].isna()) & (exp_df['abort_event'] == 3)).sum()
print("Number of rows where RTwrtStim is NaN and abort_event == 3:", count)
exp_df = exp_df[~((exp_df['RTwrtStim'].isna()) & (exp_df['abort_event'] == 3))].copy()

# comparable batch
exp_df_batch = exp_df[
    (exp_df['batch_name'] == 'Comparable') &
    (exp_df['LED_trial'].isin([np.nan, 0]))
]

# aborts and valid
df_valid_and_aborts = exp_df_batch[
    (exp_df_batch['success'].isin([1,-1])) |
    (exp_df_batch['abort_event'] == 3)
].copy()

## choice and acc columns
# 1 is right , -1 is left
df_valid_and_aborts['choice'] = df_valid_and_aborts['response_poke'].apply(lambda x: 1 if x == 3 else (-1 if x == 2 else random.choice([1, -1])))
# 1 or 0 if the choice was correct or not
df_valid_and_aborts['accuracy'] = (df_valid_and_aborts['ILD'] * df_valid_and_aborts['choice']).apply(lambda x: 1 if x > 0 else 0)

## df used for fitting - valid trials < 1s + stim
df_valid_less_than_1 = df_valid_and_aborts[
    (df_valid_and_aborts['success'].isin([1,-1])) & 
    (df_valid_and_aborts['RTwrtStim'] < 1) &
    (df_valid_and_aborts['RTwrtStim'] > 0)
]

# find ABL and ILD
ABL_arr = df_valid_and_aborts['ABL'].unique()
ILD_arr = df_valid_and_aborts['ILD'].unique()


# sort ILD arr in ascending order
ILD_arr = np.sort(ILD_arr)
ABL_arr = np.sort(ABL_arr)

print('ABL:', ABL_arr)
print('ILD:', ILD_arr)

Number of rows where RTwrtStim is NaN and abort_event == 3: 16
ABL: [10 25 40 50 55 70]
ILD: [-8.   -4.   -2.25 -1.25 -0.5   0.    0.5   1.25  2.25  4.    8.  ]


In [4]:
import numpy as np, pandas as pd, statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# -------- 1. subset to aborts & label fast (<0.3 s) ---------------
df = df_valid_and_aborts.copy()
df = df[df["abort_event"] == 3].copy()
df["fast_abort"] = (df["TotalFixTime"] < 0.300).astype(int)

# -------- 2. history regressors -----------------------------------
grp = df.groupby("session")

df["reward_prev"]   = grp["success"].shift(1).fillna(0)
win                 = 20
df["run_rate"]      = grp["success"].apply(
                        lambda s: s.rolling(win, min_periods=1).mean()
                      ).reset_index(level=0, drop=True)
df["rt_prev_abort"] = grp["TotalFixTime"].shift(1)
df["rt_prev_abort"].fillna(df["TotalFixTime"].median(), inplace=True)

# -------- 3. design matrix (no hazard term) -----------------------
predictors = ["trial", "reward_prev", "run_rate",
              "timed_ITI", "rt_prev_abort"]

scaler = StandardScaler()
df[predictors] = scaler.fit_transform(df[predictors])

X = sm.add_constant(df[predictors], has_constant="add")
y = df["fast_abort"]

# -------- 4. fit logistic GLM -------------------------------------
model = sm.GLM(y, X, family=sm.families.Binomial())
res   = model.fit()
print(res.summary())

df["p_fast"] = res.predict(X)


                 Generalized Linear Model Regression Results                  
Dep. Variable:             fast_abort   No. Observations:                10598
Model:                            GLM   Df Residuals:                    10594
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7123.3
Date:                Sat, 26 Apr 2025   Deviance:                       14247.
Time:                        17:54:58   Pearson chi2:                 1.06e+04
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007590
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.3774      0.020    -19.005

/tmp/ipykernel_32529/2214234430.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["rt_prev_abort"].fillna(df["TotalFixTime"].median(), inplace=True)


In [6]:
import pandas as pd, numpy as np, statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

results  = []                      # will collect β’s
for sid, df_sess in df_valid_and_aborts.groupby("session"):

    # ---------- 0. work on a copy so we can write cols ----------
    df_sess = df_sess.copy()

    # ---------- 1. history columns for the *whole* session -------
    # previous-trial reward (0 at session start)
    df_sess["reward_prev"] = df_sess["success"].shift(1).fillna(0)

    # running reward-rate in a 20-trial window
    df_sess["run_rate"] = (
        df_sess["success"]
              .rolling(20, min_periods=1)
              .mean()
    )

    # RT of previous abort within the session
    mask_abort             = df_sess["abort_event"] == 3
    idx_abort              = df_sess.index[mask_abort]
    df_sess.loc[idx_abort, "rt_prev_abort"] = (
        df_sess.loc[idx_abort, "TotalFixTime"].shift(1)
    )
    # fill first abort with session median RT
    df_sess["rt_prev_abort"].fillna(
        df_sess["TotalFixTime"].median(), inplace=True
    )

    # ---------- 2. keep only aborts & label fast (<0.3 s) -------
    df_ab = df_sess[mask_abort].copy()
    if len(df_ab) < 50:        # skip very short sessions
        continue

    df_ab["fast_abort"] = (df_ab["TotalFixTime"] < 0.300).astype(int)

    # ---------- 3. prepare design matrix ------------------------
    preds   = ["trial", "reward_prev", "run_rate",
               "timed_ITI", "rt_prev_abort"]
    scaler  = StandardScaler()
    Xs      = sm.add_constant(
                  scaler.fit_transform(df_ab[preds]),
                  has_constant="add"
              )
    ys = df_ab["fast_abort"]

    # ---------- 4. fit session-specific GLM ---------------------
    res = sm.GLM(ys, Xs, family=sm.families.Binomial()).fit()

    # collect coefficients
    row = dict(session=sid, **res.params.to_dict())
    results.append(row)

# ---------- 5. dataframe of per-session β-weights --------------
beta_df = pd.DataFrame(results)
print(beta_df.head())


   session     const        x1        x2        x3        x4        x5
0       72 -0.054796 -0.141482 -0.356618  0.000145 -0.180503 -0.420407
1       73 -0.616803 -0.457188 -0.584554 -0.022392  0.077513 -0.159419
2       74 -0.439095 -0.274017 -0.597019 -0.211493 -0.122100 -0.213400
3       75 -0.723997 -0.147376 -0.648721  0.050597  0.108428  0.105742
4       76 -0.932051 -0.584457 -0.667678 -0.082774 -0.011552 -0.053152


/tmp/ipykernel_32529/3083312111.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sess["rt_prev_abort"].fillna(
/tmp/ipykernel_32529/3083312111.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me